In [ ]:
!pip3 install jams
!pip install matplotlib
!pip install audiolazy
!pip install librosa
!pip3 install music21
!pip3 install MIDIUtil

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

In [ ]:
import numpy as np
import os
import pprint

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE

import scipy.io.wavfile as wav
import jams

import librosa
import audiolazy as al

import music21
from midiutil import MIDIFile


## Processing Data Input (Guitar Dataset)

In [ ]:
def timeKey(t):
    return t[0]

def loadNoteData(f):
    #f = open('/content/JamsFiles/04_Rock1-90-C_solo.jams', 'r')

   # data = f.read()
    data = jams.load(f)
    notes = []

    for i in range (0, 6):
        for j in data.annotations["note_midi"][i]["data"]:
            notes.append([j[0],j[1],j[2]])
            
    notes.sort(key=timeKey)

    #pprint.pprint(notes)
    return notes

In [ ]:
def load_audio_file(song_path, scale = False):
    srate, source_audio = wav.read(song_path)
    if scale:
        source_audio = source_audio.astype(np.float32) / max(max(source_audio),abs(min(source_audio)))

    return source_audio, srate

In [ ]:
#function to process the raw audio data.
def x_data_process(raw_data):
    scaler = preprocessing.MinMaxScaler((-1,1))
    scaled = scaler.fit_transform(raw_data)
    return scaled

In [ ]:
#TODO: Change this so it pads to match the shape of frame. not just the length
def pad(proper_size, frame):
    padsize = winSize-len(frame)
    temp = None
    if len(frame.shape) > 1:
        temp = np.zeros((frame.shape[0],padsize))
    else:
        temp = np.zeros(padsize)

    return np.concatenate((frame,temp))

In [ ]:
def songProcess(song_audio,notes,sr,hopSize = 1024,winSize = 4096,features = [],labels = []):

  testCount = 0

  offsets = np.arange(0,len(song_audio),hopSize)
  for (i,o) in enumerate(offsets):
    testCount += 1

    original_frame_size = 0

    frame = audio[o:o+winSize]

    if original_frame_size == 0:
      original_frame_size = frame.shape

    note = 0
    tiebreak = []

    #Create labels from jams file data -- if multiple notes in frame, take longest duration
    for j,(time,duration,value) in enumerate(notes):
      note_start = time*sr
      note_end = (time+duration)*sr
      if o <= note_start < o+winSize:                 #note starts in frame
        tiebreak.append(j)
      elif o <= note_end < o+winSize:                  #note ends in frame
        tiebreak.append(j)
      elif note_start < o and o+winSize <= note_end: #note continuous thru frame
        note = value

    #if multiple notes in frame, choose one that played the longest in frame
    if len(tiebreak) > 0:
      if len(tiebreak) == 1:
        note = notes[tiebreak[0]][2]
      else:
        max_dur = 0
        max_note = 0
        for index in tiebreak:
          note_start = notes[index][0] * sr
          note_duration = notes[index][1] * sr
          frame_dur = 0
          if note_start < o:
            frame_dur = note_duration+note_start-o
          elif note_start+note_duration > o+winSize:
            frame_dur = o+winSize - note_start
          else:
            frame_dur = note_duration

          if frame_dur > max_dur:
            max_dur = frame_dur
            max_note = notes[index][2]
        note = max_note


    #pad feature matrix
    if len(frame) < winSize:
      frame = pad(winSize, frame)

    #append to feature and labels
    features.append(frame)
    labels.append(round(note))  #quantize to the nearest midi value

  return features,labels,testCount

In [ ]:
def tsneFit(X, comps):
    tsne = TSNE(comps, learning_rate='auto', init='pca')
    result = tsne.fit_transform(X)
    return result

## Predict Function

In [ ]:
"""Taking the song path, trained model, and same hopSize and winSize as used in training,
Converts song into predicted midi information"""
def predict(song_path, model, hopSize, winSize):
    
    audio,sr = load_audio_file(song_path, scale=False)
    offsets = np.arange(0,len(audio),hopSize)
    x_data = []
    for (i,o) in enumerate(offsets):
        frame = audio[o:o+winSize]
        #pad
        if len(frame) < winSize:
            frame = pad(winSize, frame)

        #append to features
        x_data.append(frame)

    #preprocess data
    x_data = x_data_process(x_data)

    #column 0 is note, column 1 is time
    midi_info = np.zeros((len(offsets),2))
    raw_results = model.predict(x_data)

    for i,frame_note in enumerate(raw_results):
        midi_info[i][0] = frame_note
        midi_info[i][1] = hopSize*i/sr

        #TODO: create proper midi file from this note / time information
          #Also determine method to covert midi into sheet music

    return midi_info

### Coverting predicted MIDI info into MIDI file and Note Sheet

In [ ]:
def to_midi_sheet(midi_info,outfile):
    degrees = []

    for note, time in midi_info:
        if note != 0.0:
            degrees.append(int(note))

    track    = 0
    channel  = 0
    time     = 0    # In beats
    duration = 1    # In beats
    tempo    = 120   # In BPM
    volume   = 100  # 0-127, as per the MIDI standard

    MyMIDI = MIDIFile(1)  # One track
    MyMIDI.addTempo(track, time, tempo)

    for i, pitch in enumerate(degrees):
        MyMIDI.addNote(track, channel, pitch, time + i, duration, volume)

    with open(outfile + '.mid', "wb") as output_file:
        MyMIDI.writeFile(output_file)

    parsed = music21.converter.parse(outfile + '.mid')
    parsed.show('musicxml.png')

## Driver to load data and # of songs

In [ ]:
# LOAD SONG AND JAM FILES
song_path = r'DataSets/audio_mono-mic'
jam_path = r'DataSets/annotation'
MODE = 'solo'
inputFiles = list(zip([x for x in os.listdir(song_path) if MODE in x],[x for x in os.listdir(jam_path) if MODE in x]))

In [ ]:
##Set up features and labels for ML
numSongs = 15
counter = numSongs

notecounter = 0

hopSize = 1024
winSize = 1024

XData,YData = [],[]
XData_pre,YData_pre = [],[]
for song_file,jam_file in inputFiles:
    song = os.path.join(song_path,song_file)
    jam = os.path.join(jam_path,jam_file)
    
    audio,sr = load_audio_file(song, scale=False)
    note_info = loadNoteData(jam)

    # Can pass in a feature matrix and label array if we want to concat multiple songs together
    XData,YData,notecount = songProcess(audio,note_info,sr, hopSize, winSize)

    #Prescaled Xdata for experiement
    audio_prescaled,sr = load_audio_file(song,scale=True)
    XData_pre,YData_pre,notecount = songProcess(audio_prescaled,note_info,sr,hopSize,winSize)

    notecounter += notecount
    counter -= 1
    if counter == 0:
        break

## Neural Network Model Testing

In [ ]:
def neural_network(XTrain, XTest, YTrain, YTest, hidden_size, max_iter, solver, activation):
    #Currently, using successful values from A1, further adjustment with the processed dataset will be needed
    MLPC = MLPClassifier(hidden_layer_sizes=hidden_size, max_iter=max_iter, alpha=0.0001, 
                         learning_rate_init=0.001, solver=solver, activation=activation)
    MLPC.fit(XTrain, YTrain)

    return accuracy_score(YTest, MLPC.predict(XTest)), MLPC

In [ ]:
# NN Using No Scaling

XTrain,XTest,YTrain,YTest = train_test_split(XData,YData,test_size=0.2)
hidden_size=(60,120)
max_iter=100000
solver='adam'
activation='tanh'
accuracy, trained_model = neural_network(XTrain,XTest,YTrain,YTest,hidden_size, max_iter, solver, activation)
print("NN Model Test Accuracy:",accuracy)

song = 'DataSets/audio_mono-mic/00_BN3-119-G_solo_mic.wav'

midi_info = predict(song,trained_model,hopSize,winSize)
to_midi_sheet(midi_info,outfile="unscaled_nn_sheet_music")

NN Model Test Accuracy: 0.28050573360776243


In [ ]:
# NN Using Post Scaling
XData_postscaled = x_data_process(XData)

XTrain,XTest,YTrain,YTest = train_test_split(XData_postscaled,YData,test_size=0.2)
hidden_size=(60,120)
max_iter=100000
solver='adam'
activation='tanh'
accuracy, trained_model = neural_network(XTrain,XTest,YTrain,YTest,hidden_size, max_iter, solver, activation)
print("NN Model Test Accuracy:",accuracy)

song = 'DataSets/audio_mono-mic/00_BN3-119-G_solo_mic.wav'

midi_info = predict(song,trained_model,hopSize,winSize)
to_midi_sheet(midi_info,outfile="post_scaled_nn_sheet_music")

NN Model Test Accuracy: 0.5164657453690091


In [ ]:
# NN Using Pre Scaling
XData_pre = np.array(XData_pre)
XData_pre = XData_pre / max(XData_pre.max(), abs(XData_pre.min()))

XTrain,XTest,YTrain,YTest = train_test_split(XData_pre,YData_pre,test_size=0.2)
hidden_size=(60,120)
max_iter=10000
solver='adam'
activation='tanh'
accuracy, trained_model = neural_network(XTrain,XTest,YTrain,YTest,hidden_size, max_iter, solver, activation)
print("NN Model Test Accuracy:",accuracy)

song = 'DataSets/audio_mono-mic/00_BN3-119-G_solo_mic.wav'

midi_info = predict(song,trained_model,hopSize,winSize)
to_midi_sheet(midi_info,outfile="pre_scaled_nn_sheet_music")

NN Model Test Accuracy: 0.5802705086739194


In [ ]:
# NN Using TSNE Scaling
XData_local = x_data_process(XData)
XDataTSNE = tsneFit(XData_local, 2)

XTrain,XTest,YTrain,YTest = train_test_split(XDataTSNE,YData,test_size=0.2)
hidden_size=(60,120)
max_iter=10000
solver='adam'
activation='tanh'
accuracy, trained_model = neural_network(XTrain,XTest,YTrain,YTest,hidden_size, max_iter, solver, activation)
print("NN Model Test Accuracy:",accuracy)

song = 'DataSets/audio_mono-mic/00_BN3-119-G_solo_mic.wav'

midi_info = predict(song,trained_model,hopSize,winSize)
to_midi_sheet(midi_info,outfile="tsne_scaled_nn_sheet_music")

/Users/shawnnettleton/opt/miniconda3/envs/apress/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


NN Model Test Accuracy: 0.2935901205527786


## Support Vector Machine Model Testing

In [ ]:
def gaussianSVM(XTrain, XTest, YTrain, YTest):
    #Currently, using successful values from A2, further adjustment with the processed dataset will be needed
    clf = SVC(kernel = 'rbf', gamma = 0.01, C = 150, max_iter=100000)
    clf.fit(XTrain, YTrain)

    return accuracy_score(YTest, clf.predict(XTest)), clf

In [ ]:
def polySVM(XTrain, XTest, YTrain, YTest):
    #Currently, using successful values from A2, further adjustment with the processed dataset will be needed
    clf = SVC(kernel = 'poly', gamma = 0.01, C = 150, max_iter=100000)
    clf.fit(XTrain, YTrain)

    return accuracy_score(YTest, clf.predict(XTest)), clf

In [ ]:
def sigSVM(XTrain, XTest, YTrain, YTest):
    #Currently, using successful values from A2, further adjustment with the processed dataset will be needed
    clf = SVC(kernel = 'sigmoid', gamma = 0.01, C = 150, max_iter=100000)
    clf.fit(XTrain, YTrain)

    return accuracy_score(YTest, clf.predict(XTest)), clf


In [ ]:
def preSVM(XTrain, XTest, YTrain, YTest):
    #Currently, using successful values from A2, further adjustment with the processed dataset will be needed
    clf = SVC(kernel = 'precomputed', C = 150, max_iter=100000)
    clf.fit(XTrain, YTrain)

    return accuracy_score(YTest, clf.predict(XTest)), clf

### RBF Kernel - Gaussian SVM Testing

In [ ]:
# RBF Kernel - No Scaling

XTrain,XTest,YTrain,YTest = train_test_split(XData,YData,test_size=0.2)
accuracy, trained_model = gaussianSVM(XTrain,XTest,YTrain,YTest)
print("Gaussian SVM Model Test Accuracy:",accuracy)

song = 'DataSets/audio_mono-mic/00_BN3-119-G_solo_mic.wav'

midi_info = predict(song,trained_model,hopSize,winSize)
to_midi_sheet(midi_info,outfile="unscaled_rbf_sheet_music")

In [ ]:
# RBF Kernel - Post Scaling
XData_postscaled = x_data_process(XData)

XTrain,XTest,YTrain,YTest = train_test_split(XData_postscaled,YData,test_size=0.2)
accuracy, trained_model = gaussianSVM(XTrain,XTest,YTrain,YTest)
print("Gaussian SVM Model Test Accuracy:",accuracy)

song = 'DataSets/audio_mono-mic/00_BN3-119-G_solo_mic.wav'

midi_info = predict(song,trained_model,hopSize,winSize)
to_midi_sheet(midi_info,outfile="post_scaled_rbf_sheet_music")

In [ ]:
# RBF Kernel - Pre Scaling
XData_pre = np.array(XData_pre)
XData_pre = XData_pre / max(XData_pre.max(), abs(XData_pre.min()))

XTrain,XTest,YTrain,YTest = train_test_split(XData_postscaled,YData,test_size=0.2)
accuracy, trained_model = gaussianSVM(XTrain,XTest,YTrain,YTest)
print("Gaussian SVM Model Test Accuracy:",accuracy)

song = 'DataSets/audio_mono-mic/00_BN3-119-G_solo_mic.wav'

midi_info = predict(song,trained_model,hopSize,winSize)
to_midi_sheet(midi_info,outfile="pre_scaled_rbf_sheet_music")

In [ ]:
# RBF Kernel - TSNE Scaling
XData_local = x_data_process(XData)
XDataTSNE = tsneFit(XData_local, 2)

XTrain,XTest,YTrain,YTest = train_test_split(XDataTSNE,YData,test_size=0.2)
accuracy, trained_model = gaussianSVM(XTrain,XTest,YTrain,YTest)
print("Gaussian SVM Model Test Accuracy:",accuracy)

song = 'DataSets/audio_mono-mic/00_BN3-119-G_solo_mic.wav'

midi_info = predict(song,trained_model,hopSize,winSize)
to_midi_sheet(midi_info,outfile="tsne_scaled_rbf_sheet_music")

### Poly Kernel - Polynomial SVM Testing

### Sigmoid Kernel - Sigmoid SVM Testing